<a href="https://colab.research.google.com/github/snowysunny/Colaboratory/blob/master/tensorflow1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow 安装

In [0]:
# 不需要进行安装
#! pip install tensorflow-gpu

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [3]:
A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6],[7, 8]])
C = tf.matmul(A, B)

print(C)

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


# Tensorflow 基础

In [4]:
# Tensorflow 1 + 1
import tensorflow as tf
tf.enable_eager_execution

<function tensorflow.python.framework.ops.enable_eager_execution>

In [6]:
# 数值相加
a = tf.constant(1)
b = tf.constant(1)
c = tf.add(a, b)
print(c)

tf.Tensor(2, shape=(), dtype=int32)


In [7]:
# 矩阵相乘
A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6],[7, 8]])
C = tf.matmul(A, B)

print(C)

tf.Tensor(
[[19 22]
 [43 50]], shape=(2, 2), dtype=int32)


In [10]:
x = tf.get_variable(name='x', shape=[1], initializer=tf.constant_initializer(3.))

# 在tf.GradientTape()的上下文内，所有计算步骤都会被记录以用于求导
with tf.GradientTape() as tape:
  y = tf.square(x)  # y = x^2
  
# 计算y关于x的导数
y_grad = tape.gradient(y, x)
print([y.numpy(), y_grad.numpy()])

[array([9.], dtype=float32), array([6.], dtype=float32)]


这里x是一个初始化为3的变量(Variable)，使用tf.get_variable()声明。与普通张量一样，变量同样具有形状(shape)和类型(dtype)属性，不过使用变量需要有一个初始化过程，可以通过在tf.get_variable()中指定initializer参数来指定所使用的的初始化器。

变量和普通张量的一个重要区别是其默认能够被Tensorflow的自动求导机制所求导，因此常常被用于定义机器学习模型的参数。tf.GradientTape()是一个自动求导的记录器，在其中变量和计算步骤都会被自动记录。

**使用tf.GradientTape()计算函数$L(w, b) = ||Xw+b-y||^2$在$w=(1,2)^T, b=1$时对$w, b$的偏导数。 其中$ X=\begin{bmatrix} 1 & 2 \\ 3 & 4 \\ \end{bmatrix} , y = \begin{bmatrix} 1 \\ 2 \\ \end{bmatrix} $**